Create a 2D graph of QAOA Runtime versus N (1 to 10) for each fixed value of P from 1 to 10

（Pを１から１０の各値で固定したときの、N（１から１０）に対するQAOA Runtimeの２次元グラフを作成する。）

In [ ]:
import qokit  # QAOAのシミュレーションに使用するqokitライブラリをインポート
import numpy as np  # 数値計算用のライブラリnumpyをインポート
from itertools import combinations  # 組み合わせを生成するitertoolsモジュールからcombinationsをインポート
import scipy  # 科学技術計算用ライブラリscipyをインポート
import networkx as nx  # グラフ理論を操作するnetworkxライブラリをインポート
import time  # 時間計測用のライブラリtimeをインポート
from matplotlib import pyplot as plt  # プロット用のmatplotlibのpyplotモジュールをインポート

expectations_along_the_way = []  # QAOA実行中の期待値の推移を記録するリスト
measurements_along_the_way = []  # QAOA実行中の測定結果の推移を記録するリスト
costs = []  # カットのコストを保持するリスト

def random_graph(N, prob_connect=0.7):
    """
    N個のノードを持つランダムなグラフを生成する関数。
    prob_connectの確率でノード間にエッジを生成する。

    Parameters:
    - N: グラフのノード数
    - prob_connect: ノード間にエッジを生成する確率

    Returns:
    - A: 隣接行列
    - G: networkxのグラフオブジェクト
    """
    A = np.random.choice([0, 1], (N, N), p=[1 - prob_connect, prob_connect])
    np.fill_diagonal(A, 0)  # 自己ループを禁止
    A = np.triu(A)  # 上三角部分のみ使用
    A += A.T  # 行列を対称にする
    return (A, nx.from_numpy_array(A))

def max_cut_terms_for_graph(G):
    """
    最大カット問題の項を生成する関数。
    グラフGのエッジごとに項を生成し、最後に全結合項を追加する。

    Parameters:
    - G: networkxのグラフオブジェクト

    Returns:
    - terms: 最大カット問題の項のリスト
    """
    return list(map((lambda edge: (-0.5, edge)), G.edges)) + [((G.number_of_edges() / 2.0), ())]

def print_result(N, terms, gamma, beta, adjective=""):
    """
    QAOAの実行結果を出力する関数。
    gamma, beta, 目的関数の値を表示する。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ
    - adjective: 接頭辞として付ける形容詞（オプション）

    Returns:
    - None
    """
    p = len(gamma)
    print(f"{adjective}Gamma: {gamma}")
    print(f"{adjective}Beta: {beta}")
    print(f"{adjective}Objective: {-inv_max_cut_objective(N, p, terms)(np.hstack([gamma, beta]))}")
    print()

def evaluate_cut(G, bool_verts):
    """
    カットの数を評価する関数。
    bool_vertsで指定されたカットにおけるカットの数を数える。

    Parameters:
    - G: networkxのグラフオブジェクト
    - bool_verts: カットを表すブール値のリスト

    Returns:
    - cuts: カットの数
    """
    cuts = 0
    for (u, v) in G.edges():
        if (bool_verts[u] != bool_verts[v]):
            cuts += 1
    return cuts

def get_probabilities(N, terms, gamma, beta):
    """
    QAOAの確率分布を取得する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ

    Returns:
    - probs: 各状態の確率分布
    """
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_probabilities(_result)

def get_expectation(N, terms, gamma, beta):
    """
    QAOAの期待値を取得する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ

    Returns:
    - expectation: QAOAの期待値
    """
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_expectation(_result)

def print_probabilities(G, terms, gamma, beta, threshold=0, only_likely=False, include_cost=True):
    """
    QAOAの実行結果の確率分布を表示する関数。

    Parameters:
    - G: networkxのグラフオブジェクト
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ
    - threshold: 表示する確率の閾値
    - only_likely: 閾値を超える確率のみ表示するかどうか
    - include_cost: カットのコストを表示するかどうか

    Returns:
    - None
    """
    N = G.number_of_nodes()
    probs = get_probabilities(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), probs)):
        [index, prob] = element
        likely_str = ""
        if (prob > threshold):
            likely_str = " <- Likely outcome"

        cost_str = ""
        if (include_cost):
            contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        if (not only_likely):
            print(f"{int(index):0{N}b}{cost_str} - {prob}{likely_str}")
        else:
            if (prob > threshold):
                print(f"{int(index):0{N}b}{cost_str} - {prob}")

def get_state_vector(N, terms, gamma, beta):
    """
    QAOAの状態ベクトルを取得する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ

    Returns:
    - sv: QAOAの状態ベクトル
    """
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_statevector(_result)

def print_state_vector(G, terms, gamma, beta, include_cost=True):
    """
    QAOAの状態ベクトルを表示する関数。

    Parameters:
    - G: networkxのグラフオブジェクト
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ
    - include_cost: カットのコストを表示するかどうか

    Returns:
    - None
    """
    N = G.number_of_nodes()
    sv = get_state_vector(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), sv)):
        [index, amplitude] = element
        cost_str = ""
        if (include_cost):
            contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        print(f"{int(index):0{N}b}{cost_str} - {amplitude}")

def set_costs(G):
    """
    グラフGのカットのコストを設定する関数。

    Parameters:
    - G: networkxのグラフオブジェクト

    Returns:
    - None
    """
    costs.clear()
    N = G.number_of_nodes()
    for index in range(2**N):
        contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
        contained_verts.reverse()
        costs.append(evaluate_cut(G, contained_verts))

def inv_max_cut_objective(N, p, terms):
    """
    最大カット問題の目的関数の逆を定義する関数。

    Parameters:
    - N: グラフのノード数
    - p: QAOAのステップ数
    - terms: 最大カット問題の項のリスト

    Returns:
    - f: 目的関数の逆の関数
    """
    def f(*args):
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()

        expectation = get_expectation(N, terms, gamma, beta)
        expectations_along_the_way.append((current_time, expectation))

        if (len(costs) > 0):
            probs = get_probabilities(N, terms, gamma, beta)
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements_along_the_way.append((current_time, measurement))

        return -expectation
    return f

def optimize(N, terms, init_gamma, init_beta):
    """
    QAOAのパラメータを最適化する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - init_gamma: 初期のγパラメータ
    - init_beta: 初期のβパラメータ

    Returns:
    - gamma: 最適化されたγパラメータ
    - beta: 最適化されたβパラメータ
    """
    expectations_along_the_way.clear()
    measurements_along_the_way.clear()
    p = len(init_gamma)
    init_freq = np.hstack([init_gamma, init_beta])
    res = scipy.optimize.minimize(inv_max_cut_objective(N, p, terms), init_freq, method='COBYLA', options={'rhobeg': 0.01/N})
    gamma, beta = res.x[:p], res.x[p:]
    return (gamma, beta)

def plot_expectation(N, p, start_time):
    """
    QAOAの期待値の推移をプロットする関数。

    Parameters:
    - N: グラフのノード数
    - p: QAOAのステップ数
    - start_time: 実験開始時刻

    Returns:
    - None
    """
    def make_time_relative(input):
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations_along_the_way))
    plt.scatter(*zip(*time_relative_expectations))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("QAOA Expectation")
    plt.show()

def plot_measurements(N, p, start_time):
    """
    QAOAの測定結果の推移をプロットする関数。

    Parameters:
    - N: グラフのノード数
    - p: QAOAのステップ数
    - start_time: 実験開始時刻

    Returns:
    - None
    """
    def make_time_relative(input):
        time, measurement = input
        return (time - start_time, measurement)

    time_relative_measurements = list(map(make_time_relative, measurements_along_the_way))
    plt.scatter(*zip(*time_relative_measurements))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Measurement (best of 10)")
    plt.show()

def run_experiment(seed, N, p):
    """
    QAOAの実験を実行する関数。

    Parameters:
    - seed: 乱数シード
    - N: グラフのノード数
    - p: QAOAのステップ数

    Returns:
    - None
    """
    np.random.seed(seed)
    (A, G) = random_graph(N, 0.5)
    set_costs(G)
    solution = max(costs)
    solution_index = costs.index(solution)
    print(f"Optimal Solution: {solution_index:0{N}b} - {max(costs)}")
    vert_colors = list(map(lambda c: "green" if c == '1' else "blue", f"{int(solution_index):0{N}b}"))
    vert_colors.reverse()
    edge_colors = [("red" if vert_colors[u] != vert_colors[v] else "black") for u, v in G.edges()]

    terms = max_cut_terms_for_graph(G)
    init_gamma, init_beta = np.random.rand(2, p)

    print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    gamma, beta = optimize(N, terms, init_gamma, init_beta)
    end_time = time.time()
    print(f"Time to optimize: {end_time - start_time} seconds\n")

    print_result(N, terms, gamma, beta, "Final ")
    print_probabilities(G, terms, gamma, beta, threshold=0.5/N, only_likely=True)
    print()
    plot_expectation(N, p, start_time)
    plot_measurements(N, p, start_time)

    nx.draw(G, nx.circular_layout(G), node_color=vert_colors, edge_color=edge_colors, with_labels=True)
    return  # ここで return 文を追加する

# 実験の実行（N=10, p=5）
run_experiment(seed=0, N=10, p=5)

def set_costs(G):
    """
    グラフGのカットのコストを設定する関数。

    Parameters:
    - G: networkxのグラフオブジェクト

    Returns:
    - None
    """
    global costs  # global宣言でcostsを関数外の変数として扱う
    costs = []
    N = G.number_of_nodes()
    for index in range(2**N):
        contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
        contained_verts.reverse()
        costs.append(evaluate_cut(G, contained_verts))

def run_experiments_for_different_Ns(seed, N_values, p):
    expectations = []

    for N in N_values:
        np.random.seed(seed)
        (A, G) = random_graph(N, 0.5)
        set_costs(G)
        terms = max_cut_terms_for_graph(G)
        init_gamma, init_beta = np.random.rand(2, p)

        start_time = time.time()
        gamma, beta = optimize(N, terms, init_gamma, init_beta)
        end_time = time.time()
        print(f"N = {N}: Time to optimize = {end_time - start_time} seconds")

        expectation = get_expectation(N, terms, gamma, beta)
        expectations.append(expectation)

    return N_values, expectations

# Nの値を設定
N_values = [1,2,3,4,5,6,7,8,9,10,11]

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 1  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()


# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 2  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 3  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 4  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 5  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 6  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 7  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 8  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 9  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 10  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title(f'QAOA Expectation vs. N (p={p})')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()